In [1]:
import graphlab as gl
import pandas as pd

In [2]:
target_pd = pd.read_csv('../data/modeling/LANDSCAPE/target_data_LANDSCAPE_4979.csv', delimiter='|')

In [3]:
target_pd.shape

(4979, 12)

In [4]:
target_pd.columns

Index([u'owner', u'id', u'user_is_pro', u'user_can_buy_pro',
       u'user_total_views', u'image_ncomments', u'image_nfavs', u'image_nsets',
       u'image_npools', u'image_views', u'image_ntags', u'image_tags'],
      dtype='object')

In [5]:
target_pd.drop('image_tags', axis=1, inplace=True)

In [6]:
target_gl = gl.SFrame(data=target_pd)

This non-commercial license of GraphLab Create for academic use is assigned to mpmakris@gmail.com and will expire on June 30, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1469921461.log


In [7]:
images_raw = gl.image_analysis.load_images('../data/images/LANDSCAPE/')

Read 344 images in 5.00636 secs	speed: 68.7125 file/sec

Read 681 images in 10.0144 secs	speed: 67.2924 file/sec

Read 1016 images in 15.0349 secs	speed: 66.7257 file/sec

Read 1360 images in 20.0398 secs	speed: 68.7328 file/sec

Read 1699 images in 25.0455 secs	speed: 67.7224 file/sec

...

Read 42567 images in 626.176 secs	speed: 68.3902 file/sec

Read 42906 images in 631.189 secs	speed: 67.6173 file/sec

Read 43232 images in 636.218 secs	speed: 64.8277 file/sec

Read 43564 images in 641.221 secs	speed: 66.3616 file/sec

Read 43904 images in 646.226 secs	speed: 67.9258 file/sec

In [ ]:
images = images_raw

In [8]:
images['filename'] = images['path'].apply(lambda x: x[x.find('/')+1:])

Using default 16 lambda workers.

To maximize the degree of parallelism, add the following code to the beginning of the program:

"graphlab.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 40)"

Note that increasing the degree of parallelism also increases the memory footprint.

In [9]:
check = True
while check:
    images['filename'] = images['filename'].apply(lambda x: x[x.find('/')+1:])
    if images[0]['filename'].find('/') < 0:
        check = False

In [10]:
images['owner'] = images['filename'].apply(lambda x: x[:x.find('_')])
images['id'] = images['filename'].apply(lambda x: x[x.find('_')+1:])
images['id'] = images['id'].apply(lambda x: x[:x.find('.')])

In [13]:
images['id'] = images['id'].apply(lambda x: int(x))

In [14]:
images = images.join(target_gl, on=['owner', 'id'], how='inner')

In [15]:
images.save('LANDSCAPE_images_feature_and_target')

In [16]:
images.head()

path,image,filename,owner,id,user_is_pro
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 1601 Width: 1067,13342549@N08_7539228342.jpg ...,13342549@N08,7539228342,0
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 1065 Width: 1600,35006702@N00_26600332142.jpg ...,35006702@N00,26600332142,1
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 1066 Width: 1600,73199107@N05_25064238174.jpg ...,73199107@N05,25064238174,0
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 1200 Width: 1600,48587703@N00_16424970432.jpg ...,48587703@N00,16424970432,0
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 860 Width: 1600,128817646@N07_24454808223.jpg ...,128817646@N07,24454808223,0
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 687 Width: 1024,8031838@N08_24751545659.jpg ...,8031838@N08,24751545659,1
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 778 Width: 1600,52019166@N02_17426816153.jpg ...,52019166@N02,17426816153,0
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 1067 Width: 1600,9477623@N03_24273987665.jpg ...,9477623@N03,24273987665,1
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 1139 Width: 1600,89806999@N08_26907876823.jpg ...,89806999@N08,26907876823,1
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 1202 Width: 1600,28527690@N07_26526226645.jpg ...,28527690@N07,26526226645,1


In [17]:
images['resized_image'] = gl.image_analysis.resize(images['image'], 800, 600, channels=3)

In [18]:
extractor = gl.feature_engineering.DeepFeatureExtractor(features='resized_image', output_column_prefix='deep_')

Downloading https://static.turi.com/products/graphlab-create/resources/models/python2.7/imagenet_model_iter45/dir_archive.ini to /var/tmp/graphlab-ubuntu/4470/262be342-3361-4d00-bbd1-a64b455334fa.ini

Downloading https://static.turi.com/products/graphlab-create/resources/models/python2.7/imagenet_model_iter45/objects.bin to /var/tmp/graphlab-ubuntu/4470/8e1b67f8-4ac1-4339-ab65-ad051ba0af4b.bin

In [19]:
extractor = extractor.fit(images)

In [20]:
extracted_model = extractor['model']

In [21]:
images = extractor.transform(images)

Images being resized.

### Save Data, then Train-Test-Split

In [22]:
#images.save('./LANDSCAPE_images_ready_to_model.save')
images = gl.load_sframe('./LANDSCAPE_images_ready_to_model.save')

In [23]:
images_train, images_test = images.random_split(0.8)

### Make Models

In [25]:
images_train.head()

path,image,filename,owner,id,user_is_pro
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 1601 Width: 1067,13342549@N08_7539228342.jpg ...,13342549@N08,7539228342,0
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 1065 Width: 1600,35006702@N00_26600332142.jpg ...,35006702@N00,26600332142,1
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 1066 Width: 1600,73199107@N05_25064238174.jpg ...,73199107@N05,25064238174,0
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 1200 Width: 1600,48587703@N00_16424970432.jpg ...,48587703@N00,16424970432,0
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 860 Width: 1600,128817646@N07_24454808223.jpg ...,128817646@N07,24454808223,0
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 1067 Width: 1600,9477623@N03_24273987665.jpg ...,9477623@N03,24273987665,1
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 1139 Width: 1600,89806999@N08_26907876823.jpg ...,89806999@N08,26907876823,1
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 1202 Width: 1600,28527690@N07_26526226645.jpg ...,28527690@N07,26526226645,1
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 900 Width: 1600,64233859@N06_22241370296.jpg ...,64233859@N06,22241370296,1
/home/ubuntu/efs/GIT/Photo_Pro/data/images/LAN ...,Height: 596 Width: 1024,86696434@N05_26683049506.jpg ...,86696434@N05,26683049506,0


In [26]:
model_regression_imageviews = gl.linear_regression.create(images_train, target='image_views', features=['deep_.resized_image'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Linear regression:

--------------------------------------------------------

Number of examples          : 3780

...

| 6         | 14       | 1.000000  | 2.600285     | 47312.046300       | 15098.103022         | 2146.493334   | 1526.627041     |

| 10        | 18       | 1.000000  | 3.258500     | 41024.701261       | 13906.720159         | 1841.779074   | 1785.469245     |

+-----------+----------+-----------+--------------+--------------------+----------------------+---------------+-----------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [30]:
model_classifier_ispro = gl.classifier.create(images_train, target='user_is_pro', features=['deep_.resized_image'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: LogisticClassifier, SVMClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.


WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 3781

...

| 10        | 18       | 1.000000  | 2.126899     | 0.695054          | 0.637255            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

PROGRESS: Model selection based on validation accuracy:
PROGRESS: ---------------------------------------------
PROGRESS: LogisticClassifier              : 0.656863
PROGRESS: SVMClassifier                   : 0.637255
PROGRESS: ---------------------------------------------
PROGRESS: Selecting LogisticClassifier based on validation set performance.


In [72]:
model_svmclassifier_ispro = graphlab.svm_classifier.create(images_train, target='user_is_pro',
                                                           features=['deep_.resized_image'], penalty=0.1,
                                                           solver='auto', feature_rescaling=True,
                                                           convergence_threshold=0.01, lbfgs_memory_level=11,
                                                           max_iterations=100, class_weights=None,
                                                           validation_set='auto', verbose=True)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

SVM:

--------------------------------------------------------

Number of examples          : 3781

...

| 90        | 100      | 1.000000  | 16.315332    | 0.989685          | 0.578431            |

| 100       | 110      | 1.000000  | 18.110026    | 0.991272          | 0.558824            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [61]:
model_logitclassifier_ispro = graphlab.logistic_classifier.create(images_train, target='user_is_pro',
                                                                  features=['deep_.resized_image'],
                                                                  l2_penalty=0.01, l1_penalty=0.0, solver='auto',
                                                                  feature_rescaling=True, convergence_threshold=0.01,
                                                                  step_size=1.0, lbfgs_memory_level=11,
                                                                  max_iterations=100, class_weights=None,
                                                                  validation_set='auto', verbose=True)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 3793

...

| 90        | 110      | 1.000000  | 18.267615    | 1.000000          | 0.526042            |

| 100       | 124      | 1.000000  | 20.460531    | 1.000000          | 0.526042            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

### Evaluate Models

In [36]:
gl.linear_regression.LinearRegression.evaluate(model_regression_imageviews, dataset=images_test, metric='auto')

{'max_error': 35472.05024965469, 'rmse': 2710.0482875816465}

In [65]:
# images_test_predictions = model_classifier_ispro.classify(images_test)
classifier_results = model_classifier_ispro.evaluate(images_test)
svmclassifier_results = model_svmclassifier_ispro.evaluate(images_test)
logitclassifier_results = model_logitclassifier_ispro.evaluate(images_test)

In [71]:
print "SVM CLASSIFIER:"
print "Recall: {}".format(svmclassifier_results['recall'])
print "Precision: {}".format(svmclassifier_results['precision'])
print "Accuracy: {}".format(svmclassifier_results['accuracy'])

SVM CLASSIFIER:
Recall: 0.571129707113
Precision: 0.565217391304
Accuracy: 0.582494969819


In [74]:
gl.canvas.set_target('ipynb')
classifier_results['roc_curve'].show(view='Scatter Plot', x='fpr', y='tpr')
#svmclassifier_results['roc_curve'].show(view='Scatter Plot', x='fpr', y='tpr')
logitclassifier_results['roc_curve'].show(view='Scatter Plot', x='fpr', y='tpr')
#classifier_results['roc_curve'].show(view='Scatter Plot', x='threshold', y='tpr')

### Summaries:

In [64]:
model_classifier_ispro.summary()
model_svmclassifier_ispro.summary()
model_logitclassifier_ispro.summary()

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 4097
Number of examples             : 3781
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 4096

Hyperparameters
---------------
L1 penalty                     : 0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 10
Solver status                  : TERMINATED: Iteration limit reached.
Training time (sec)            : 2.3102

Settings
--------
Log-likelihood                 : 1947.4746

Highest Positive Coefficients
-----------------------------
deep_.resized_image[3080]      : 0.1554
deep_.resized_image[2069]      : 0.1106
deep_.resized_image[3285]      : 0.0943
deep_.resized_image[882]       : 0.0872
deep_.resized_image[1949]      : 0.0867

Lowest Negative Coefficients
----------------------------
deep_.resized_image[2545]      : -0.1239


### Save Models

In [73]:
model_classifier_ispro.save('model_classifier_ispro')
model_svmclassifier_ispro.save('model_svmclassifier_ispro')
model_logitclassifier_ispro.save('model_logitclassifier_ispro')

### Neural Net Model

In [86]:
network = gl.deeplearning.create(images_train, target='user_is_pro', features=['deep_.resized_image'])
network.layers

layer[0]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 10
layer[1]: SigmoidLayer
layer[2]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 2
layer[3]: SoftmaxLayer

In [137]:
cust_network_layers = list()
#cust_network_layers.append(gl.deeplearning.layers.FlattenLayer())
cust_network_layers.append(gl.deeplearning.layers.FullConnectionLayer(100, init_sigma=0.5))
cust_network_layers.append(gl.deeplearning.layers.RectifiedLinearLayer())
cust_network_layers.append(gl.deeplearning.layers.DropoutLayer(0.5))
cust_network_layers.append(gl.deeplearning.layers.FullConnectionLayer(200, init_sigma=.1))
cust_network_layers.append(gl.deeplearning.layers.TanhLayer())
cust_network_layers.append(gl.deeplearning.layers.FullConnectionLayer(150, init_sigma=.1))
cust_network_layers.append(gl.deeplearning.layers.TanhLayer())
cust_network_layers.append(gl.deeplearning.layers.FullConnectionLayer(2, init_sigma=0.5))
cust_network_layers.append(gl.deeplearning.layers.SoftmaxLayer())

custom_network = gl.deeplearning.NeuralNet()
#custom_network.params['max_iterations'] = 20
custom_network.layers = cust_network_layers
custom_network.verify(input_shape=[1, 1, 4096], output_shape=2)

True

In [138]:
model_neuralnetclassifier_ispro = gl.neuralnet_classifier.create(images_train, target='user_is_pro',
                                                                 features=['deep_.resized_image'],
                                                                 network=custom_network, max_iterations=100,
                                                                 validation_set=images_test, batch_size=1000,
                                                                 model_checkpoint_path='neural_network_model',
                                                                 model_checkpoint_interval=5
                                                                 )

Using network:

### network layers ###
layer[0]: FullConnectionLayer
  init_sigma = 0.5
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 100
layer[1]: RectifiedLinearLayer
layer[2]: DropoutLayer
  threshold = 0.5
layer[3]: FullConnectionLayer
  init_sigma = 0.1
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 200
layer[4]: TanhLayer
layer[5]: FullConnectionLayer
  init_sigma = 0.1
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 150
layer[6]: TanhLayer
layer[7]: FullConnectionLayer
  init_sigma = 0.5
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 2
layer[8]: SoftmaxLayer
### end network layers ###

### network parameters ###
model_checkpoint_interval = 5
learning_rate = 0.001
batch_size = 1000
momentum = 0.9
model_checkpoint_path = neural_network_model
### end network parameters ###



For Nvidia GPU CUDA support, please go to https://turi.com/download/install-graphlab-create-gpu.html

Creating neuralnet using cpu

Training with batch size = 1000

+-----------+----------+--------------+-------------------+---------------------+-----------------+

...

| 45        | 4000     | 145.736171   | 0.537250          | 0.525151            | 1248.123291     |

Checkpointing model into neural_network_model

| 46        | 4000     | 149.099084   | 0.536750          | 0.528169            | 1189.495117     |

[ERROR] graphlab.toolkits._main: Toolkit error: NN Iteration Cancelling.


ToolkitError: NN Iteration Cancelling.

In [ ]:
model_neuralnetclassifier_ispro.extract_features() # WHAT IS THIS?!

In [116]:
neuralnetclassifier_results = model_neuralnetclassifier_ispro.evaluate(images_test)

In [117]:
neuralnetclassifier_results.keys()

['confusion_matrix', 'accuracy']

In [119]:
neuralnetclassifier_results['accuracy']

0.5663983821868896

In [118]:
neuralnetclassifier_results['confusion_matrix']

target_label,predicted_label,count
0,0,326
1,0,241
0,1,190
1,1,237


In [120]:
nn_test_predictions = model_neuralnetclassifier_ispro.predict(images_test)

In [121]:
type(nn_test_predictions)

graphlab.data_structures.sarray.SArray

In [ ]:
gl.deeplearning.get_builtin_neuralnet('imagenet')